In [4]:
from __future__ import annotations
#from PIL import Image
#import scipy.stats as st
import sys
sys.path.append(sys.path[0]+"/..")
from SYS_ATL import proc, Procedure

In [5]:
def gen_blur():
    @proc
    def blur(n: size, m: size, k_size: size,
             image: R[n, m] @ IN, kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
        for i in par(0, n):
            for j in par(0, m):
                res[i, j] = 0.0
        for i in par(0, n):
            for j in par(0, m):
                for k in par(0, k_size):
                    for l in par(0, k_size):
                        if i+k >= 1 and i+k-n < 1 and j+l >= 1 and j+l-m < 1:
                            res[i, j] += kernel[k, l] * image[i+k-1, j+l-1]

    return blur

In [6]:
blur = gen_blur()
blur

```python
def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i in par(0, n):
        for j in par(0, m):
            res[i, j] = 0.0
    for i in par(0, n):
        for j in par(0, m):
            for k in par(0, k_size):
                for l in par(0, k_size):
                    if i + k >= 1 and i + k - n < 1 and j + l >= 1 and j + l - m < 1:
                        res[i, j] += kernel[k, l] * image[i + k - 1, j + l - 1]

```

In [7]:
blur.reorder('k', 'l')

```python
def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i in par(0, n):
        for j in par(0, m):
            res[i, j] = 0.0
    for i in par(0, n):
        for j in par(0, m):
            for l in par(0, k_size):
                for k in par(0, k_size):
                    if i + k >= 1 and i + k - n < 1 and j + l >= 1 and j + l - m < 1:
                        res[i, j] += kernel[k, l] * image[i + k - 1, j + l - 1]

```

In [8]:
blur.split('j',4,['j1','j2'])

```python
def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i in par(0, n):
        for j1 in par(0, m / 4):
            for j2 in par(0, 4):
                if 4 * j1 + j2 < m:
                    res[i, 4 * j1 + j2] = 0.0
    for i in par(0, n):
        for j1 in par(0, m / 4):
            for j2 in par(0, 4):
                if 4 * j1 + j2 < m:
                    for k in par(0, k_size):
                        for l in par(0, k_size):
                            if i + k >= 1 and i + k - n < 1 and 4 * j1 + j2 + l >= 1 and 4 * j1 + j2 + l - m < 1:
                                res[i, 4 * j1 +
                                    j2] += kernel[k, l] * image[i + k - 1,
                                                                4 * j1 + j2 +
                                                                l - 1]

```

In [9]:
blur.split('i[1]',4,['i1','i2'])

```python
def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i1 in par(0, n / 4):
        for i2 in par(0, 4):
            if 4 * i1 + i2 < n:
                for j in par(0, m):
                    res[4 * i1 + i2, j] = 0.0
    for i in par(0, n):
        for j in par(0, m):
            for k in par(0, k_size):
                for l in par(0, k_size):
                    if i + k >= 1 and i + k - n < 1 and j + l >= 1 and j + l - m < 1:
                        res[i, j] += kernel[k, l] * image[i + k - 1, j + l - 1]

```

In [10]:
blur = blur.split('j',4,['j1','j2'])
blur = blur.unroll('j2')
blur

```python
def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i in par(0, n):
        for j1 in par(0, m / 4):
            if 4 * j1 + 0 < m:
                res[i, 4 * j1 + 0] = 0.0
            if 4 * j1 + 1 < m:
                res[i, 4 * j1 + 1] = 0.0
            if 4 * j1 + 2 < m:
                res[i, 4 * j1 + 2] = 0.0
            if 4 * j1 + 3 < m:
                res[i, 4 * j1 + 3] = 0.0
    for i in par(0, n):
        for j1 in par(0, m / 4):
            if 4 * j1 + 0 < m:
                for k in par(0, k_size):
                    for l in par(0, k_size):
                        if i + k >= 1 and i + k - n < 1 and 4 * j1 + 0 + l >= 1 and 4 * j1 + 0 + l - m < 1:
                            res[i, 4 * j1 +
                                0] += kernel[k, l] * image[i + k - 1,
                                                           4 * j1 + 0 + l - 1]
            if 4 * j1 + 1 < m:
                for k in par(0, k_size):
                    for l in par(0, k_size):
                        if i + k >= 1 and i + k - n < 1 and 4 * j1 + 1 + l >= 1 and 4 * j1 + 1 + l - m < 1:
                            res[i, 4 * j1 +
                                1] += kernel[k, l] * image[i + k - 1,
                                                           4 * j1 + 1 + l - 1]
            if 4 * j1 + 2 < m:
                for k in par(0, k_size):
                    for l in par(0, k_size):
                        if i + k >= 1 and i + k - n < 1 and 4 * j1 + 2 + l >= 1 and 4 * j1 + 2 + l - m < 1:
                            res[i, 4 * j1 +
                                2] += kernel[k, l] * image[i + k - 1,
                                                           4 * j1 + 2 + l - 1]
            if 4 * j1 + 3 < m:
                for k in par(0, k_size):
                    for l in par(0, k_size):
                        if i + k >= 1 and i + k - n < 1 and 4 * j1 + 3 + l >= 1 and 4 * j1 + 3 + l - m < 1:
                            res[i, 4 * j1 +
                                3] += kernel[k, l] * image[i + k - 1,
                                                           4 * j1 + 3 + l - 1]

```